# Video Moderation - detecting inappropriate stored videos

You can use Amazon Rekognition to detect content that is inappropriate, unwanted, or offensive. You can use Rekognition moderation APIs in social media, broadcast media, advertising, and e-commerce situations to create a safer user experience, provide brand safety assurances to advertisers, and comply with local and global regulations.

Amazon Rekognition Video inappropriate or offensive content detection in stored videos is an asynchronous operation. To start detecting inappropriate or offensive content, call [StartContentModeration](https://docs.aws.amazon.com/rekognition/latest/APIReference/API_StartContentModeration.html). Amazon Rekognition Video publishes the completion status of the video analysis to an Amazon Simple Notification Service topic. If the video analysis is successful, call [GetContentModeration](https://docs.aws.amazon.com/rekognition/latest/APIReference/API_GetContentModeration.html) to get the analysis results. For more information about starting video analysis and getting the results, see [Calling Amazon Rekognition Video operations](https://docs.aws.amazon.com/rekognition/latest/dg/api-video.html). 

In this tutorial, we will show you how to use Amazon Rekognition Moderation API to moderate stored videos. And how to moderate text in a video using Amazon Rekognition and Amazon Comprehend.

![image-moderation-with-text-arc](../images/video-moderation-with-text.png)

- [Step 1: Setup Notebook](#step1)
- [Step 2: Moderate video using Rekognition video moderation API](#step2)
- [Step 3: Moderate text in video](#step3)
- [Step 4: Clean up](#step4)

# Step 1: Setup Notebook <a id="step1"></a>

In [ ]:
# First, let's get the latest installations of our dependencies
%pip install --upgrade pip
%pip install botocore --upgrade
%pip install boto3 --upgrade
%pip install -U botocore

In [ ]:
%pip install IPython --upgrade

In [ ]:
import boto3
import sagemaker as sm
import os
import io
import datetime
from IPython.display import Video
from IPython.display import HTML, display
import uuid
import json
import time

# variables
data_bucket = sm.Session().default_bucket()
region = boto3.session.Session().region_name

os.environ["BUCKET"] = data_bucket
os.environ["REGION"] = region
role = sm.get_execution_role()

print(f"SageMaker role is: {role}\nDefault SageMaker Bucket: s3://{data_bucket}")

s3=boto3.client('s3', region_name=region)
rekognition=boto3.client('rekognition', region_name=region)
sagemaker = boto3.client('sagemaker', region_name=region)
comprehend = boto3.client('comprehend', region_name=region)

For this lab, we will moderate a demo video containing a few typical inappropriate scenes, such as alcohol, tobacco, gambling, and suggestive. The video locates at `datasets/moderation-video.mp4`.
![video-moderation-data](../images/video-moderation-dataset.png)

Now, let's upload the video to the default S3 bucket for Rekognition to access.

In [ ]:
s3_key = 'content-moderation-im/video-moderation/moderation-video.mp4'
s3.upload_file('../datasets/moderation-video.mp4', data_bucket, s3_key)

# Step 2: Moderate video using Rekognition moderation API  <a id="step2"></a>
Call Rekognition **StartContentModeration** API to detect inappropriate information in the video. Rekognition Video moderation API is an asynchronize API that will start a job managed by Rekognition. We will receive a job_id back when we start the job. Then use iteration logic to heart-beat the **GetContentModeration** API to check job status until the job completes.
The below process should take ~1 minute to complete.

However, it is highly recommended to check out the Notification Channel feature which allows you to receive a SNS notification when the detection job is completed instead of periodically polling the status of the detection. Check [boto3 document](https://docs.aws.amazon.com/cli/latest/reference/rekognition/start-content-moderation.html) for details.

In [ ]:
startModerationLabelDetection = rekognition.start_content_moderation(
    Video={
        'S3Object': {
            'Bucket': data_bucket,
            'Name': s3_key,
        }
    }
)

moderationJobId = startModerationLabelDetection['JobId']
display("Job Id: {0}".format(moderationJobId))

getContentModeration = rekognition.get_content_moderation(
    JobId=moderationJobId,
    SortBy='TIMESTAMP'
)

while(getContentModeration['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')

    getContentModeration = rekognition.get_content_moderation(
    JobId=moderationJobId,
    SortBy='TIMESTAMP')

display(getContentModeration['JobStatus'])

Let's display the video and labels in HTML.

In [ ]:
s3_video_url = s3.generate_presigned_url('get_object', Params={'Bucket': data_bucket, 'Key': s3_key})
video_tag = f"<video id='cccvid1' controls='controls' width='640' height='360' name='Video' src='{s3_video_url}'></video>"

label_html = ''''''
for label in getContentModeration["ModerationLabels"]:
    if len(label["ModerationLabel"]["ParentName"]) > 0:
        label_html += f'''<a onclick="document.getElementById('cccvid1').currentTime={round(label['Timestamp']/1000)}">[{label['Timestamp']} ms]: 
                {label['ModerationLabel']['Name']}, confidence: {round(label['ModerationLabel']['Confidence'])}</a><br/>
                '''
display(HTML(video_tag))

Run the below file to print out all the detected labels. Click on the item to navigate to the above video's timestamp.

In [ ]:
display(HTML(label_html))

# Step 3: Moderate text in video <a id="step3"></a>

Text detection with Amazon Rekognition Video is an asynchronous operation. You start text detection by calling **StartTextDetection** which returns a job identifier (JobId ) When the text detection operation finishes, Amazon Rekognition publishes a completion status to the Amazon Simple Notification Service topic registered in the initial call to **StartTextDetection**. To get the results of the text detection operation, first check that the status value published to the Amazon SNS topic is SUCCEEDED . if so, call **GetTextDetection** and pass the job identifier (JobId ) from the initial call of **StartLabelDetection**.

The below process should take ~1 minute to complete.

In [ ]:
getTextDetection = rekognition.start_text_detection(
    Video={
        'S3Object': {
            'Bucket': data_bucket,
            'Name': s3_key,
        }
    }
)

textJobId = getTextDetection['JobId']
display("Job Id: {0}".format(moderationJobId))

getTextDetection = rekognition.get_text_detection(JobId=textJobId)

while(getTextDetection['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')

    getTextDetection = rekognition.get_text_detection(JobId=textJobId)

display(getTextDetection['JobStatus'])

Now, let's display the video and use the hyperlinks to navigate the timestamps where text was detected.

In [ ]:
s3_video_url = s3.generate_presigned_url('get_object', Params={'Bucket': data_bucket, 'Key': s3_key})
video_tag = f"<video id='cccvid2' controls='controls' width='640' height='360' name='Video' src='{s3_video_url}'></video>"

text_html = ''''''
for txt in getTextDetection["TextDetections"]:
    if txt["TextDetection"]["Type"] == 'LINE':
        text_html += f'''<a onclick="document.getElementById('cccvid2').currentTime={round(txt['Timestamp']/1000)}">[{txt['Timestamp']} ms]: 
                {txt["TextDetection"]["DetectedText"]}, confidence: {round(txt["TextDetection"]["Confidence"],2)}</a><br/>
                '''
display(HTML(video_tag))

Run the below file to print out all the detected texts. Click on the item to navigate to the above video's timestamp.

In [ ]:
display(HTML(text_html))

After extracting texts from the video, we can then moderate the texts using Amazon Comprehend. 

### [Amazon Comprehend](https://aws.amazon.com/comprehend/) is a managed AI service for Nature Language Processing and text analysis.
You can use the Amazon Comprehend console or APIs to detect personally identifiable information (PII) in English text documents. PII is a textual reference to personal data that could be used to identify an individual. 

In [ ]:
detected_text_list=[]
textDetections=getTextDetection['TextDetections']
for text in textDetections:
    if text["TextDetection"]['Type'] == 'LINE' and text["TextDetection"]['DetectedText'] not in detected_text_list:
        detected_text_list.append(text["TextDetection"]['DetectedText'])
    
    
for text in detected_text_list:
    response=comprehend.detect_pii_entities(Text=text, LanguageCode='en')
    if len(response['Entities']) > 0: 
        print ('Detected text:' + text)
        print("Detected PII entity is: " + text[response['Entities'][0]['BeginOffset']:response['Entities'][0]['EndOffset']])
        print("PII type is: " + response['Entities'][0]['Type'])
        print("Confidence score is: " + str(response['Entities'][0]['Score']))
        print()

# Step 4: Clean up <a id="step4"></a>

Remove the resources that may incur the cost

In [ ]:
# remove the image from s3 bucket
response = s3.delete_object(Bucket=data_bucket, Key=s3_key)
print(response)

# Conclusion
In this lab, we moderated a video (containing text) using Amazon Rekognition and Amazon Comprehend.